## CSV 파일 읽기

In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForMaskedLM, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Load the uploaded CSV files to check their structure and contents
file_path_projections = "C:/Users/PC/desktop/바탕 화면/archive/indiana_projections.csv"
file_path_reports ="C:/Users/PC/desktop/바탕 화면/archive/indiana_reports.csv"

# Read the CSV files
data_projections = pd.read_csv(file_path_projections)
data_reports = pd.read_csv(file_path_reports)

## Frontal" 프로젝션 데이터만 필터링, 데이터와 병합

In [2]:
# Filter for "Frontal" images in the projections dataset
frontal_data = data_projections[data_projections['projection'] == 'Frontal']

# Merge the frontal data with the reports data using 'uid' as the key
merged_data = pd.merge(frontal_data, data_reports, on='uid', how='inner')

# Check the resulting merged data
merged_data.head()


,uid,filename,projection,MeSH,Problems,image,indication,comparison,findings,impression
0,1,1_IM-0001-4001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,2_IM-0652-1001.dcm.png,Frontal,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,3_IM-1384-1001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,4_IM-2050-1001.dcm.png,Frontal,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,5_IM-2117-1003002.dcm.png,Frontal,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.


## BERT 모델을 위한 데이터 준비와 Masked Language Modeling (MLM) 데이터셋 클래스를 정의

In [3]:
# Merge the frontal data with the reports data using 'uid' as the key
merged_data = pd.merge(frontal_data, data_reports, on='uid', how='inner')

# Check the resulting merged data
merged_data.head()
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.utils.data import DataLoader, Dataset
# findings와 impression 결합
merged_data["report"] = merged_data["findings"].fillna("") + " " + merged_data["impression"].fillna("")
merged_data["report"] = merged_data["report"].str.strip()

# 필요한 컬럼 선택 및 NaN 제거
merged_data = merged_data[["uid", "filename", "projection", "indication", "report"]].dropna()

# BERT Tokenizer 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# MLM 데이터셋 클래스 정의
class MLM_Dataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        
        # BERT 토크나이징
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        input_ids = encoding["input_ids"].squeeze(0)
        attention_mask = encoding["attention_mask"].squeeze(0)

        # 랜덤 마스킹 적용 (MLM 방식)
        labels = input_ids.clone()
        rand = torch.rand(input_ids.shape)
        mask_arr = (rand < 0.15) * (input_ids != tokenizer.cls_token_id) * (input_ids != tokenizer.sep_token_id) * (input_ids != tokenizer.pad_token_id)
        input_ids[mask_arr] = tokenizer.mask_token_id

        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


In [4]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')


# 데이터셋 및 DataLoader 생성
mlm_dataset = MLM_Dataset(merged_data["report"].tolist() + merged_data["indication"].tolist(), tokenizer)
mlm_dataloader = DataLoader(mlm_dataset, batch_size=2, shuffle=True)

# 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForMaskedLM.from_pretrained("bert-base-uncased").to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll ge

In [5]:
import random

# 'report' 컬럼에서 'XXXX'가 포함된 문장만 필터링
filtered_texts = [text for text in merged_data["report"].dropna() if "XXXX" in text]

# 랜덤 샘플 선택 (filtered_texts가 비어 있으면 기본값 제공)
sample_text = random.choice(filtered_texts) if filtered_texts else "No valid text found."

# 마스킹 수행
masked_text = sample_text.replace("XXXX", tokenizer.mask_token)

# 토크나이징 및 텐서 변환
inputs = tokenizer(masked_text, return_tensors="pt").to(device)
mask_idx = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]

# MLM 예측 수행
with torch.no_grad():
    predictions = model(**inputs).logits

# 마스크된 토큰 예측 및 복원
predicted_text = masked_text
for idx in mask_idx.tolist():
    predicted_token = tokenizer.decode(torch.argmax(predictions[0, idx]))
    predicted_text = predicted_text.replace(tokenizer.mask_token, predicted_token, 1)

# 결과 출력
print("=" * 50)
print("📝 원본 텍스트:\n", sample_text, "\n", "=" * 50)
print("\n🔍 마스킹된 텍스트:\n", masked_text, "\n", "=" * 50)
print("\n🎯 예측된 텍스트:\n", predicted_text, "\n", "=" * 50)

📝 원본 텍스트:
 There is a 1 cm nodule within one of the lung bases, seen only on the lateral view. There is a calcified right hilar lymph node and right granuloma. Heart size is normal. No pneumothorax. 1 cm nodule within the lung base, seen only on the lateral view. Consider imaging correlation with XXXX abdomen study, if available. . 

🔍 마스킹된 텍스트:
 There is a 1 cm nodule within one of the lung bases, seen only on the lateral view. There is a calcified right hilar lymph node and right granuloma. Heart size is normal. No pneumothorax. 1 cm nodule within the lung base, seen only on the lateral view. Consider imaging correlation with [MASK] abdomen study, if available. . 

🎯 예측된 텍스트:
 There is a 1 cm nodule within one of the lung bases, seen only on the lateral view. There is a calcified right hilar lymph node and right granuloma. Heart size is normal. No pneumothorax. 1 cm nodule within the lung base, seen only on the lateral view. Consider imaging correlation with the abdomen study, if ava

In [6]:
from torch.optim import AdamW
from transformers import BertForMaskedLM
import torch
from torch.utils.data import DataLoader

In [7]:
# Create DataLoader for training
# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Define number of epochs
epochs = 1

# Training loop
model.train()


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
from tqdm import tqdm

for epoch in range(epochs):
    total_loss = 0
    
    # tqdm을 사용하여 배치 진행 상태 표시
    for batch in tqdm(mlm_dataloader, desc=f"Epoch {epoch + 1}/{epochs}", leave=False):
        # Move the batch to the device (GPU/CPU)
        batch = {k: v.to(device) for k, v in batch.items()}

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass through the model
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass (compute gradients)
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Accumulate the loss for reporting
        total_loss += loss.item()

    # Average loss for this epoch
    avg_loss = total_loss / len(mlm_dataloader)
    print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}")


Epoch 1/1:   5%|███▌                                                              | 198/3733 [10:06<3:00:39,  3.07s/it]

In [ ]:
# After training, you can evaluate the model on some validation or test set if you have one, 
# but since you only have the dataset, let's check the loss at the end.

# Model evaluation mode
model.eval()

# Perform a test step with a single batch to check if everything works after training
with torch.no_grad():
    for batch in mlm_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        print("Test batch loss: ", outputs.loss.item())
        break  # Stop after one batch for test